# Unity Catalogクイックスタート(SQL)

Unity Catalogを用いることで、以下の機能でDatabricks上の資産に対するガバナンスを強化することができます。

- **データアクセス制御**
  - 一度定義するだけで、全てのワークスペース、全ての言語、全てのユースケースに対してセキュリティを適用
- **監査機能**
  - ガバナンスユースケースに対応するために、全てのクエリーに対するきめ細かい監査を実現
- **データリネージ**
  - テーブル、カラムに対するデータリネージの自動収集
- **データ探索**
  - お使いのレイクハウスにおいてデータ利用者が信頼できるデータを検索するためのインタフェースを提供
- **属性ベースのアクセス制御**
  - 行列レベルのネイティブなセキュリティ、タグによるポリシー適用


このノートブックでは、以下の方法を説明することで、Unity Catalogを使用し始めるサンプルワークフローを提供します。

- カタログを選択し、新規スキーマを作成する。
- マネージドテーブルを作成しスキーマに追加する。
- 3レベルの名前空間を用いてテーブルにクエリーを実行する。
- データリネージを追跡する。
- テーブルに対するデータアクセス権を管理する。

## 3レベルの名前空間

Unity Catalogはデータを整理するために3レベルの名前空間、カタログ、スキーマ(データベースとも呼ばれます)、テーブルとビューを提供します。テーブルを参照するには以下の文法を使用します。

>`<catalog>.<schema>.<table>`

![3_level_namespace.png](./3_level_namespace.png "3_level_namespace.png")

### 新規カタログの作成

それぞれのUnity Catalogメタストアには、空のスキーマ`default`を含む`workspace`というデフォルトのカタログが含まれています。

新規カタログを作成するには`CREATE CATALOG`コマンドを使用します。新規カタログを作成するには、メタストア管理者である必要があります。

以降のコマンドでは、以下の方法を説明します。

- 新規カタログの作成
- カタログの選択
- 全てのカタログの表示
- カタログに対するアクセス権の許可
- カタログの全てのアクセス権の表示

**注意** 以下では`uc_quickstart_catalog`というカタログを作成します。

カタログやスキーマにアクセスするには、`USE`コマンドを用いて明示的にカタログやスキーマを指定する必要があります。指定されない場合にはデフォルトのカタログやスキーマが使用されます。

In [0]:
SELECT current_catalog();

In [0]:
--- 新規カタログの作成
CREATE CATALOG IF NOT EXISTS uc_quickstart_catalog;

In [0]:
-- カタログの選択
USE CATALOG uc_quickstart_catalog;

In [0]:
--- メタストアの全てのカタログの表示
SHOW CATALOGS;

In [0]:
--- アカウントの全ユーザーに対してCREATE SCHEMA, USE CATALOG権限を許可
--- これは、他のアカウントレベルのグループ、ユーザーにも動作します
GRANT CREATE SCHEMA, USE CATALOG
ON CATALOG uc_quickstart_catalog
TO `account users`;

In [0]:
--- カタログのアクセス権の確認
SHOW GRANT ON CATALOG uc_quickstart_catalog;

### スキーマの作成、管理
データベースとも呼ばれるスキーマは、Unity Catalog名前空間の2番目のレイヤーです。これらはテーブルとビューを論理的に整理します。

In [0]:
--- カタログに新規スキーマを作成
CREATE SCHEMA IF NOT EXISTS quickstart_schema
COMMENT "quickstart_schemaという新規Unity Catalogのスキーマです";

In [0]:
-- 選択されたカタログのスキーマを表示
SHOW SCHEMAS;

In [0]:
-- スキーマの詳細を表示
DESCRIBE SCHEMA EXTENDED quickstart_schema;

In [0]:
-- スキーマの削除(以下の行のコメントを解除して試してください。ノートブックの以降のコマンドを実行するには、スキーマを再作成してください)
-- DROP SCHEMA quickstart_schema CASCADE

### マネージドテーブルの作成

*マネージドテーブル*は、Unity Catalogでテーブルを作成する際のデフォルトの方法となります。`LOCATION`が指定されない場合、テーブルはメタストアに設定されたマネージドの格納場所に作成されます。

以下のコマンドでは、次の方法を説明します。

- スキーマの選択
- マネージドテーブルを作成し、レコードをインサート
- スキーマの全テーブルの表示
- テーブルの詳細の表示

In [0]:
-- スキーマの選択
USE quickstart_schema;

In [0]:
-- マネージドのDeltaテーブルを作成、2行を追加
CREATE TABLE IF NOT EXISTS quickstart_table
  (columnA Int, columnB String) PARTITIONED BY (columnA);

INSERT INTO TABLE quickstart_table
VALUES
  (1, "one"),
  (2, "two");

In [0]:
-- スキーマの全テーブルの表示
SHOW TABLES IN quickstart_schema;

上のコマンドを実行した結果の2行目は一旦スルーしてください。詳細は[こちら](https://docs.databricks.com/aws/ja/notebooks/notebooks-code#sql-%E3%82%BB%E3%83%AB%E3%81%AE%E7%B5%90%E6%9E%9C%E3%82%92%E8%AA%BF%E3%81%B9%E3%82%8B)をご覧ください。

In [0]:
-- テーブルの詳細を表示
DESCRIBE TABLE EXTENDED quickstart_table;

3レベルの名前空間を用いることで、いくつかの方法でテーブルにアクセスすることができます。

- 完全修飾名によるアクセス
- デフォルトカタログを選択し、スキーマ、テーブル名を用いてテーブルにアクセス
- デフォルトスキーマを選択し、テーブル名を使用

以下の3つのコマンドは機能的に等価です。

In [0]:
-- 3レベル名前空間を用いたテーブルへのクエリーの実行
SELECT
  *
FROM
  uc_quickstart_catalog.quickstart_schema.quickstart_table;

In [0]:
-- デフォルトカタログを選択し、スキーマ、テーブル名でテーブルへクエリーを実行
USE CATALOG uc_quickstart_catalog;
SELECT *
FROM quickstart_schema.quickstart_table;

In [0]:
-- デフォルトカタログとデフォルトスキーマを選択し、テーブル名でテーブルへクエリーを実行
USE CATALOG uc_quickstart_catalog;
USE quickstart_schema;
SELECT *
FROM quickstart_table;

#### テーブルの削除

`DROP TABLE`コマンドで*マネージドテーブル*を削除すると、背後のデータファイルも削除されます。

`DROP TABLE`コマンドで*外部テーブル*を削除すると、テーブルに関するメタデータはカタログから削除されますが、背後のデータファイルは削除されません。

In [0]:
-- マネージドテーブルの削除(以下の行のコメントを解除して試してください。ノートブックの以降のコマンドを実行するには、テーブルを再作成してください)
-- DROP TABLE uc_quickstart_catalog.quickstart_schema.quickstart_table

## リネージの追跡

Unity Catalogで管理されるテーブルはリネージ(系統情報)が追跡されます。どのテーブルからどのテーブルが作成されたのか、カラムレベルで追跡が可能です。

[Unity Catalogを使用してデータリネージを表示](https://docs.databricks.com/aws/ja/data-governance/unity-catalog/data-lineage)

In [0]:
-- スキーマを作成し、権限を付与します
CREATE SCHEMA lineagedemo;
GRANT USE SCHEMA, CREATE TABLE ON SCHEMA lineagedemo TO `account users`;

In [0]:
-- スキーマの全テーブルの表示
SHOW TABLES IN uc_quickstart_catalog.lineagedemo

In [0]:
--DROP TABLE uc_quickstart_catalog.lineagedemo.dinner;
--DROP TABLE uc_quickstart_catalog.lineagedemo.menu;

In [0]:
-- ソーステーブルを作成します
CREATE TABLE IF NOT EXISTS uc_quickstart_catalog.lineagedemo.menu (
  recipe_id INT,
  app string,
  main string,
  desert string
);

INSERT INTO uc_quickstart_catalog.lineagedemo.menu 
    (recipe_id, app, main, desert) 
VALUES 
    (1,"Ceviche", "Tacos", "Flan"),
    (2,"Tomato Soup", "Souffle", "Creme Brulee"),
    (3,"Chips","Grilled Cheese","Cheescake");

In [0]:
-- ソーステーブルから派生テーブルを作成します
CREATE TABLE uc_quickstart_catalog.lineagedemo.dinner 
AS SELECT recipe_id, concat(app," + ", main," + ",desert) as full_menu FROM uc_quickstart_catalog.lineagedemo.menu

<a href="/explore/data/uc_quickstart_catalog/lineagedemo" target="_blank">カタログエクスプローラ</a>でリネージを確認します。

## データに対するアクセス権の管理

データへのアクセスを管理するために、`GRANT`と`REVOKE`文を使用します。Unity Catalogはデフォルトでセキュリティ保護を行い、データへのアクセスは自動では許可されません。最初は、全てのユーザーがデータにアクセスすることができません。メタストアの管理者とデータのオーナーが、アカウントレベルのユーザーとグループに対してアクセス権の許可、剥奪を行うことができます。

[Unity Catalog権限とセキュリティ保護可能なオブジェクト](https://docs.databricks.com/aws/ja/data-governance/unity-catalog/manage-privileges/privileges)

#### オーナーシップ

Unity Catalogのそれぞれのオブジェクトにはオーナーが存在します。*プリンシパル*と呼ばれるアカウントレベルのユーザー、グループがオーナーになることができます。プリンシパルはセキュリティ保護可能なオブジェクトを作成する際、あるいは`ALTER`文を用いてオーナーシップが転送された際にオーナーになります。

### アクセス権の管理

Unity Catalogのオブジェクトに対して以下のアクセス権を許可することができます。
- `ALL PRIVILEGES`: セキュリティ保護可能なオブジェクトとその子オブジェクトに適用されるすべての権限を、明示的に指定せずに付与または取り消すために使用されます。
- `SELECT`: テーブル、ビューからデータを読み込むことを許可します。
- `MODIFY`: テーブルへの行追加、更新、データの削除を許可します。
- `CREATE CATALOG/SCHEMA/TABLE`: カタログ、スキーマにセキュリティ保護可能な子供のオブジェクトを作成することを許可します。
- `USAGE CATALOG/SCHEMA`: このアクセス権はセキュリティ保護可能なオブジェクト自身へのアクセスを許可しませんが、子供のオブジェクトにアクセスするためにセキュリティ保護可能オブジェクトを通過することを許可します。例えば、テーブルからデータを取得するためには、ユーザーは当該テーブルに対する`USAGE`権限と、親のスキーマ、親のカタログに対する`USAGE`権限を必要とします。特定のグループに対してデータの名前空間のセクションへのアクセスを制限するために、この権限を使用することができます。

権限は子供のセキュリティ保護可能なオブジェクトに**継承されます**。

[権限継承へのアップグレード](https://docs.databricks.com/aws/ja/data-governance/unity-catalog/manage-privileges/upgrade-privilege-model)

以下のコマンドでは、次の方法を説明します。

- アクセス権の許可
- セキュリティ保護可能オブジェクトに対するアクセス権の表示
- アクセス権の剥奪

#### アクセス権の許可

セキュリティ保護可能オブジェクトにプリンシパルのアクセス権を許可します。メタストア管理者とセキュリティ保護可能オブジェクトのオーナーのみがアクセス権を許可することができます。

In [0]:
-- スキーマの USE を許可
GRANT USE SCHEMA
ON SCHEMA quickstart_schema
TO `account users`;

In [0]:
-- プリンシパルに対してテーブルの　SELECT を許可
GRANT SELECT
ON TABLE quickstart_schema.quickstart_table
TO `account users`;

#### アクセス権の表示

セキュリティ保護可能なオブジェクトに許可された全てのアクセス権の一覧を表示します。

In [0]:
-- quickstart_tableのアクセス権の表示
SHOW GRANTS
ON TABLE uc_quickstart_catalog.quickstart_schema.quickstart_table;

In [0]:
-- quickstart_schemaのアクセス権の表示
SHOW GRANTS
ON SCHEMA uc_quickstart_catalog.quickstart_schema;

#### アクセス権の剥奪

これまで許可されていたセキュリティ保護可能なオブジェクトのアクセス権をプリンシパルから剥奪します。

In [0]:
REVOKE SELECT
ON TABLE quickstart_schema.quickstart_table
FROM `account users`;

In [0]:
-- quickstart_tableのアクセス権の表示
SHOW GRANTS
ON TABLE uc_quickstart_catalog.quickstart_schema.quickstart_table;

## クリーンアップ

以下の`DROP CATALOG ... CASCADE`でカタログおよび内部のスキーマ、テーブルを削除します。

In [0]:
DROP CATALOG uc_quickstart_catalog CASCADE;

# END